In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("/moto/eaton/projects/macaques/metadata.csv")
df[["Species", "Group", "SRR", "BioSample", "Sample", "Study", "PRJ"]]

,Species,Group,SRR,BioSample,Sample,Study,PRJ
0,Macaca mulatta northern,mulatta,SRR4454026,SAMN05883679,SRS1762015,SRP092140,PRJNA345528
1,Macaca mulatta southern low altitude,mulatta,SRR4454020,SAMN05883709,SRS1762009,SRP092140,PRJNA345529
2,Macaca mulatta southern high altitude,mulatta,SRR4453966,SAMN05883736,SRS1761955,SRP092140,PRJNA345530
3,Macaca mulatta Indian,mulatta,SRR5628058,SAMN07168901,SRS2238957,SRP049547,PRJNA251548
4,Macaca fascicularis northern,fascicularis,fasno,SAMN00116341,SRS117874,SRP045755,PRJNA51411
5,Macaca fascicularis southern,fascicularis,fasso,SAMD00006158,DRS000787,DRP000438,PRJDB2038
6,Macaca fuscata,mulatta,DRR002233,SAMD00011919,DRS001583,DRP000620,PRJDB2459
7,Macaca fuscata,mulatta,fuscata2,SAMD00013516,DRS002017,DRP000657,PRJDB2648
8,Macaca thibethana,sinica,SRR1024051,SAMN02390221,SRS498543,SRP032525,PRJNA226187
9,Macaca assamensis,sinica,SRR2981114,SAMN04316321,SRS1196892,SRP067118,PRJNA305009


In [4]:
%%bash
source ~/.bashrc
conda activate py2
for i in Chr19 Chr18 Chr17; do
    bcftools filter -e 'FORMAT/DP<10 | FORMAT/GQ<30 | FORMAT/GT="0/1"' --set-GTs . --threads 12 /moto/eaton/projects/macaques/calls/$i.raw.vcf -O v -o /moto/eaton/projects/macaques/calls/$i.GQDP.vcf
done

In [5]:
%%bash
source ~/.bashrc
conda activate py2
for i in Chr19 Chr18 Chr17; do
    vcftools --recode --vcf /moto/eaton/projects/macaques/calls/$i.GQDP.vcf --minGQ 30 --minDP 10 --max-missing 0.4 --out /moto/eaton/projects/macaques/calls/$i.GQDP.maxmiss
done


VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--vcf /moto/eaton/projects/macaques/calls/Chr19.GQDP.vcf
	--minDP 10
	--minGQ 30
	--max-missing 0.4
	--out /moto/eaton/projects/macaques/calls/Chr19.GQDP.maxmiss
	--recode

After filtering, kept 20 out of 20 Individuals
Outputting VCF file...
After filtering, kept 5730831 out of a possible 6071317 Sites
Run Time = 1281.00 seconds

VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--vcf /moto/eaton/projects/macaques/calls/Chr18.GQDP.vcf
	--minDP 10
	--minGQ 30
	--max-missing 0.4
	--out /moto/eaton/projects/macaques/calls/Chr18.GQDP.maxmiss
	--recode

After filtering, kept 20 out of 20 Individuals
Outputting VCF file...
After filtering, kept 6035209 out of a possible 6213641 Sites
Run Time = 1345.00 seconds

VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--vcf /moto/eaton/projects/macaques/calls/Chr17.GQDP.vcf
	--minDP 

In [ ]:
%%bash
source ~/.bashrc
conda activate vcfkit
for i in Chr19 Chr18 Chr17; do
    vk phylo fasta /moto/eaton/projects/macaques/calls/$i.GQDP.maxmiss.recode.vcf > /moto/eaton/projects/macaques/calls/$i.GQDP.maxmiss.fasta
done

In [ ]:
%%bash
source ~/.bashrc
conda activate py2
export LC_ALL="en_US.UTF-8"
for i in Chr19 Chr18 Chr17; do
    perl /moto/eaton/projects/macaques/scripts/fasta2relaxedPhylip.pl -f /moto/eaton/projects/macaques/calls/$i.GQDP.maxmiss.fasta -o /moto/eaton/projects/macaques/calls/$i.GQDP.maxmiss.phy
done

In [ ]:
%%bash
source ~/.bashrc
conda activate base
export LC_ALL="en_US.UTF-8"
for i in Chr19 Chr18 Chr17; do
    python /moto/eaton/users/nsl2119/raxml_ascbias/ascbias.py -p /moto/eaton/projects/macaques/calls/$i.GQDP.maxmiss.phy -o /moto/eaton/projects/macaques/calls/$i.GQDP.maxmiss.raxml.phy
done

In [ ]:
%%bash
source ~/.bashrc
conda activate py2
export LC_ALL="en_US.UTF-8"
for i in Chr19 Chr18 Chr17; do
    raxmlHPC-PTHREADS-SSE3 -s /moto/eaton/projects/macaques/calls/$i.GQDP.maxmiss.raxml.phy -m ASC_GTRGAMMA --asc-corr=lewis -f a -T 12 -N 100 -n $i -o SRR8285768 -w /moto/eaton/projects/macaques/calls/ -x 12345 -p 12345
done

In [ ]:
%%bash
source ~/.bashrc
conda activate test
export LC_ALL="en_US.UTF-8"
iqtree -s /moto/eaton/projects/macaques/calls/Chr20.GQDP.maxmiss.raxml.phy -o SRR8285768 -st DNA -m MFP+ASC -bb 5000 -alrt 5000 -nt 12 -mem 96G -redo

In [ ]:
%%bash
iqtree -s /moto/eaton/projects/macaques/calls/vcftoolsfilterinvardel.phy -m MFP+ASC -bb 5000 -alrt 5000 -nt 12 -mem 96gb

In [ ]:
iqtree -s /moto/eaton/projects/macaques/mapped/Chr19/calls/vcftoolsfilter.phy -m MFP+ASC -nt 12 -n 10000 -mem 64gb

In [ ]:
##on desktop
scp nsl2119@moto.rcs.columbia.edu:/moto/eaton/projects/macaques/calls/RAxML_bestTree.vcftoolsfilter /home/nicolas/Desktop/